In [1]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [2]:
platform=os.getenv('OS_PLATFORM', 'demo8')
display_platform(platform)
inventory = read_inventory("$HOME/env/{}_hosts.ini".format(platform))
conn = connectToCloud(platform)

In [3]:
show_notebook_url(platform, host_ip="10.3.216.210", port=8888)

#### Servers status (ping servers in 'ping_check' group)

In [4]:
display_html_ping_all(inventory, False)

#### Show VMs (servers)

In [5]:
showServerList( conn )

6 flavors
37 images
1 servers


name,status,flavor,image,addresses
karimtestvm,ACTIVE,m1.small[2],debian-wheezy-amd64-20140929-disk[44d35d8c-cc29-4134-a342-dd92507f3add],default-net172.0.100.53


#### Show server port status

In [6]:
display_html_ping_ports_all(inventory)

ping(seeduc[10.3.160.11:22]) ... ping(seed[10.3.160.10:22]) ... ping(seedvm[10.3.160.6:22]) ... 

seeduc[10.3.160.11:22],OK
seedvm[10.3.160.6:22],TIMEOUT
seed[10.3.160.10:22],OK


#### Show Endpoint URLs status (check ports open)

In [7]:
display_html_endpoint_urls(conn)

10.3.160.26:8774 [compute],OK
10.3.160.26:9292 [image],OK
10.3.160.26:5000 [identity],OK
10.3.160.26:8776 [volume],OK
10.3.160.26:9696 [network],OK
